Create Flag parameter

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')

Creating Dimension Model

Creating dimension model

Fetch Realtive Columns

In [0]:
df_scr=spark.sql("""SELECT  DISTINCT(DriveTrain) FROM parquet.`abfss://silver@carsmalyanadatalake.dfs.core.windows.net/cars_sales`""")

dim_make Sink - Initial and incremental

In [0]:
    
if spark.catalog.tableExists('cars.gold.dim_DriveTrain'):    
    df_sink=spark.sql("""SELECT dim_DriveTrain_key, Model
     FROM cars_cat.gold.dim_DriveTrain
     """)

   
    
else:
     df_sink=spark.sql("""SELECT 1 as dim_DriveTrain_key, DriveTrain
     FROM parquet.`abfss://silver@carsmalyanadatalake.dfs.core.windows.net/cars_sales`
     where 1=0""")



Filtering new records and old records

In [0]:
df_filter=df_scr.join(df_sink, df_scr.DriveTrain==df_sink.DriveTrain, 'left').select(df_scr.DriveTrain,df_sink.dim_DriveTrain_key)

In [0]:
df_filter.display()

DriveTrain,dim_DriveTrain_key
Rear,null
null,null
All,null
Front,null


df_filter _old

In [0]:
df_filter_old=df_filter.filter(df_filter.dim_DriveTrain_key.isNotNull())

In [0]:
df_filter_old.display()

DriveTrain,dim_DriveTrain_key


In [0]:
df_filter_new=df_filter.filter(df_filter.dim_DriveTrain_key.isNull())

Craete Suurogate Key

FETCH the max suurogate key from existing table

In [0]:
if incremental_flag=='0':

    max_value=0
else:
    max_value_df=spark.sql("""SELECT MAX(dim_DriveTrain_key) FROM cars.gold.dim_DriveTrain""")
    max_value=max_value_df.collect()[0][0]



Create Surrogate column and add the max surrogate key 

In [0]:
from pyspark.sql import functions as F


In [0]:
df_filter_new=df_filter_new.withColumn('dim_DriveTrain_key',max_value+F.monotonically_increasing_id())

In [0]:
df_filter_new.createOrReplaceTempView('df_filter_new')

In [0]:
df_filter_new.display()

DriveTrain,dim_DriveTrain_key
Rear,0
null,1
All,2
Front,3


Create final df = df_filter_old+ df_filter_new

In [0]:
df_final=df_filter_old.union(df_filter_new)

SCD TYPE -1 (UPSERT)

In [0]:
from delta.tables import DeltaTable


In [0]:
if spark.catalog.tableExists('cars_cat.gold.dim_DriveTrain'): 
    delta_table=Deltatable.forPath(spark,"abfss://gold@carsmalyanadatalake.dfs.core.windows.net/dim_DriveTrain")

    delta_table.alias("trg").merge(df_final.alias("src"),"trg.dim_DriveTrain_key=src.dim_DriveTrain_key").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
        df_final.write.format('delta').mode('overwrite').option("path","abfss://gold@carsmalyanadatalake.dfs.core.windows.net/dim_DriveTrain").saveAsTable("cars_cat.gold.dim_DriveTrain")